In [16]:
#輸入你想爬的板，並設定範圍
start = 39421
base_format = "https://www.ptt.cc/bbs/Gossiping/index{index}.html"
page = 10000

In [17]:
import requests
from bs4 import BeautifulSoup

targets = []
for p in range(page):
    response = requests.get(base_format.format(index=str(start - p)),
                            cookies={"over18":"1"})
    html = BeautifulSoup(response.text, "html.parser")
    entries = html.find_all("div", class_="r-ent")
    for e in entries:
        t = e.find("div", class_="title").find("a")
        if t is not None:
            title = t.text
            if title.startswith("[") and not title.startswith("[公告]"):
                targets.append((title, t["href"]))
                
print("總共:", len(targets), "篇文章")
print("前十篇:", targets[:10])

總共: 125352 篇文章
前十篇: [('[問卦] 大家都用沐浴乳洗澡還是肥皂？', '/bbs/Gossiping/M.1574760749.A.A08.html'), ('[問卦] 2399歐元的洗衣機在歐洲算貴嗎？', '/bbs/Gossiping/M.1574760793.A.083.html'), ('[問卦] 如果圍統神的屁孩突然鳴槍怎麼辦？', '/bbs/Gossiping/M.1574760804.A.9B4.html'), ('[問卦] 第一次刺青要刺甚麼?', '/bbs/Gossiping/M.1574760822.A.EEC.html'), ('[問卦] 慟！！！人為什麼會偷吃？ ', '/bbs/Gossiping/M.1574760838.A.92B.html'), ('[新聞] 礦業法細說從頭：為什麼拖？為什麼急？', '/bbs/Gossiping/M.1574760843.A.4E9.html'), ('[問卦] 台灣有會騎馬的人嗎?', '/bbs/Gossiping/M.1574760872.A.A62.html'), ('[問卦] 中柱智障怎麼那麼多？', '/bbs/Gossiping/M.1574760882.A.B6E.html'), ('[新聞] 韓國瑜冒最新金句「得民調得痔瘡」', '/bbs/Gossiping/M.1574761030.A.651.html'), ('[問卦] 大家希望曾博恩訪問韓什麼問題？', '/bbs/Gossiping/M.1574761078.A.339.html')]


In [18]:
#依序存成Json檔
def save_article(article, dn):
    article_base = "https://www.ptt.cc"
    article_url = article_base + article
    response = requests.get(article_url, cookies={"over18":"1"})
    html = BeautifulSoup(response.text, "html.parser")

    main = html.find("div", id="main-content")
    metas = main.find_all("span", class_="article-meta-value")
    user_id = metas[0].text.split(" ")[0]
    title = metas[2].text
    pattern = re.compile(r"\[.+\]")
    article_type = (re.match(pattern, title)
               .group()
               .replace("[", "")
               .replace("]", ""))
    article_title = re.split(pattern, title)[-1].lstrip(" ")
    article_time = str(datetime.strptime(metas[3].text, 
                                         "%a %b %d %H:%M:%S %Y"))

    pushes = main.find_all("div", class_="push")
    article_push = []
    article_score = 0

    for m in main.find_all("div", class_="article-metaline"):
        m.extract()
    for m in main.find_all("div", class_="article-metaline-right"):
        m.extract()    

    for p in pushes:
        tag = p.find("span", class_="push-tag")
        if "推" in tag.text:
            push_score = 1
            article_score = article_score + 1
        elif "噓" in tag.text:
            push_score = -1
            article_score = article_score - 1
        else:
            push_score = 0
        push_id = p.find("span", class_="push-userid").text
        push_content = p.find("span", class_="push-content").text
        article_push.append({"type":push_score,
                             "id":push_id,
                             "content":push_content})
        p.extract()

    ip = None
    for f in main.find_all("span", class_="f2"):
        if "發信站:" in f.text:
            pat = re.compile("[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+")
            ip = re.search(pat, f.text).group()
            f.extract()
        elif "文章網址:" in f.text:
            f.extract()

    data = {
        "ptt_id":user_id,
        "post_url":article_url,
        "post_type":article_type,
        "post_title":article_title,
        "post_time":article_time,
        "post_score":article_score,
        "post_pushes":article_push,
        "post_ip":ip,
        "post_content":main.text
    }

    fn = article_url.split("/")[-1].split(".html")[0]
    with open(os.path.join(dn, fn + ".json"), "w", encoding="utf-8") as f:
        json.dump(data, f)

In [ ]:
import re
from datetime import datetime
import json
import os

dn = "E:/pttdata"
if not os.path.exists(dn):
    os.makedirs(dn)

for t in targets:
    try:
        save_article(t[1], dn)
    except Exception as e:
        print(e)

'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'group'
list index out of range
list index out of range
'NoneType' object has no attribute 'group'
list index out of range
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'text'
list index out of range
list index out of range
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
list index out of range
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'text'
list index out of range
list index out of range
list index out of range
list index out of range
time data 'Tue Nov 19 20:16:4